<b>

<p>
<center>
<font size="5">

</font>
</center>
</p>

<p>
<center>
<font size="5">
COVID-19 Open Research Dataset Challenge (CORD-19)
</font>
</center>
</p>

<p>
<center>
<font size="4">
Authors: Chao Zhou, Ruijin Jia, Matteo Bucalossi
</font>
</center>
</p>

<p>
<center>
<font size="3">
Machine Learning I (DATS 6202), Spring 2020
</font>
</center>
</p>

</b>

### Introduction

Last month, in response to the global COVID-19 pandemic, the White House and other leading research institutions, including the Allen Institute for AI, have prepared the COVID-19 Open Research Dataset (CORD-19): this dataset includes over 57,000 scholarly articles, most of them with full text, about COVID-19, SARS-CoV-2, and related coronaviruses. As literature published keeps increasing in scale, the medical community would need AI tools to quickly gain insights and directions to fight this disease in a timely manner.  
Kaggle has issued a [competition](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) to develop such tools to help the medical community for this high priority scientific challenge. The public dataset represents a highly readable and clean collection of materials, and it is beeing updated constantly as more publications are released and added to the corpus.  

Thanks to the minimal work on pre-processing required by this dataset, we decided to apply a Transformer model to extract sentence embeddings for machine learning task-specific "heads". Given that the pre-trained transformer by the UKPLab uses BERT, we decided to train and fine-tune the same transformer on SciBERT by Allen AI to obtain more relevant embeddings for a scientific and medical corpus.  
We then used said embeddings to train different clustering algorithms for unsupervised learning: we used UAMP for dimensionality reduction, and HDBSCAN for density-based clustering while LDA for probability-based clustering. Also, we integrated a semantic search algorithm based on cosine similarity that can take user questions (we recommend to take ideas from [Tasks](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/tasks)) and provides the top 5 relevant articles from the corpus.

# Data Preprocessing

Luckily the dataset had already been cleaned for the most part, and we only had to create an hashable corpus of text from json files. Once we had the body text and the abstact for each article, and its relevant metadata, in a dataframe, we cleaned the text columns of odd characters using regex.  Here we call the [preprocessing](https://github.com/matteobucalossi50/CORD-19-Challenge/blob/master/scripts/preprocessing.py) script to obtain a clean dataframe to use for our machine learning tasks.

In [ ]:
### load preprocessing.py
### print dataframe.head or something

# Train Sentence Transformer on sciBERT

### Transformers

When it comes to natural language processing, the most recent developments have seen attention mechanisms prevailing versus more traditiona RNN models. These models use an architecture called Transformer, much faster and easier to parallelize than other networks. This is where these models revolutionize the field: an attention mechanism looks at an input sequence and decides at each step which other components of the sequence are important<sup>[1](https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04)</sup> - meaning it can replicate the way we actually process text, i.e. not only focusing on single words but also considering what's around it to make sense of the language.   

Transformers are architectures for transforming a sequence into another by using Encoder and Decoder; yet, they only imply attention mechanisms without any Recurrent Networks (previously the go-to models for many NLP tasks). Here's an image<sup>[2](https://arxiv.org/abs/1706.03762)</sup>  to illustrate such Transformer architecture, with the Encoder part on the left and the Decoder on the right.
![](images/Transformers_scheme.png)
We won't bore you with the details of the model and its mathematical aspects, but we can point out two main characteristics of Transformers:
 - the Multi-Head Attention layers treat each word's relationship with every other word in the same sentence, basically paying attention to more words than just one when processing sequences. These layers will apply to every (input/target, depending on encoder/decoder) sentence the following equation: ![](images/scaled dot-prod attention.png)
 - the positional encodings of words are dense vectors (some extra word embeddings) representing the position of a word within the sentencem and are added to each word's embeddings.  



### BERT & SciBERT

A 2018 paper<sup>[3](https://arxiv.org/abs/1810.04805)</sup> published by various Google researchers brought to life a nowadays state-of-the-art application of a Transformer-based architectures for self-supervised pretraining on large corpus, BERT (*Bidirectional Encoder Representations from Transformers*).  BERT is a method of pre-training language representations, so that we can train a general-purpose model on an immense corpus and then use said model for downstream NLP tasks - its bidirectional characteristic allows to represent each word within its context (i.e. other words in the sentence, both on the left and right of represented word).

If the original BERT trained a large (12-layer to 24-layer) Transformer on a large corpuse of Wikipedia and BookCorpus, more recent BERT-alike models have trained the same architecture on specific corpuses for domain-relevant tasks. For instance, in 2019 Allen AI released SciBERT,<sup>[4](https://arxiv.org/abs/1903.10676)</sup> a BERT model trained on huge corpus of scientific papers (82% of which from biomedical domain)  from [semanticscholar.org](https://www.semanticscholar.org/), which significantly improves BERT performance on downstream NLP tasks specific to scientific problems. We believe that using SciBERT for this project will yield much better results than the generic original BERT model given the specificity of our data.

### Sentence-BERT

BERT-alike models described above have set a new bar for sentence-pair regressions tasks, but unfortunately they need both sentences to be fed in the Transformer, causing such a computational overhead that makes unsupervised tasks virtually impossible.  
Thus, in 2019 researchers at UKPLab released SBERT,<sup>[5](https://arxiv.org/abs/1908.10084)</sup> a modification of pre-trained BERT to derive semantic sentence embeddings easily comparable for similarity and clustering. SBERT fine-tunes BERT-alike models with a siamese or triplet network structure to obtain such embeddings. Such a revolutionary paper proposed a model that still maintains BERT-level accuracy, while scaling down time complexity from 65 hours to 5 seconds for specific unsupervised NLP tasks, including similarity comparison, semantic search and clustering. (ah! exactly what we are trying to do here!)  
Here's an illustration of the SBERT architecture as described in the paper (the structure would not change if the objective function was different, as it may be for different tasks): ![](images/SBERT architecture.png) 

# Sentence embeddings for analysis

Now, we intend to perform some clustering as well as semantic search on the CORD-19 dataset. To do so, we need SBERT sentence embeddings so that we can accomplish these tasks in reasonable time. We decided that pre-trained embeddings on BERT would have not provide the SOTA outcome we were looking for, thus we fine-tuned our sentence embedding method on SciBERT to get science-specific sentence embeddings.  
The SBERT library provides the code to tune any BERT-like model (in our case, SciBERT)on the Natural Language Inference (NLI) data, published by Allen AI. The following script trained SciBERT on the NLI dataset using a Softmax Classifier as training loss and the STS (Semantic Text Similarity) dataset as benchmark for evaluation, and provided us a file-tuned sentence embedder to derive embeddings for our clustering and search tasks on the CORD-19 dataset.

In [ ]:
## import training_scibert.py

At this point, we can simply use our fine-tuned model (perfect for our biomedical dataset) to encode our corpus. This script will provide the embeddings we need, both for the abstract and separately for the full text of each article. We will then access these embeddings in the dataframe for downstream tasks below.

In [ ]:
## import embeddings.py

# Clustering 

### UAMP 

blabla

In [6]:
## call clustering.uamp 

## HDBSCAN

blabla

In [7]:
## call clustering.hdbscan 

### visualization / evaluation

## K-means

blabla

In [8]:
## call clustering.kmeans 

### visualization / evaluation

## LDA

blabla

In [ ]:
## call clustering.lda 

### visualization / evaluation

# Semantic search

### Cosine similarity
An amazing use of the embeddings we obtained with our fine-tuned Sentence Transformer model is to query the corpus and find the most similar embeddings to the query's embeddings. This can be simply done by calculating cosine similarity among embeddings, and then select the least-distant ones as the most relevant to the query.  
Here we propose a semantic search system where the user can input a question about COVID-19 and they will get the top 5 most relevant articles from the corpus. We suggest to interrogate our system with questions taken from the Kaggle's suggested [tasks](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/tasks).

In [ ]:
## call searches and get table out

# Conclusion

blablabla 